In [1]:
import time
import pickle
import gzip

import numpy as np
import open3d as o3d
import open3d.visualization as vis

def save_gzip(path, data):
    with gzip.open(path, 'wb') as f:
        pickle.dump(data, f)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
device_id = 0

config = o3d.io.read_azure_kinect_sensor_config(f"configs/config_{device_id}.json")
sensor = o3d.io.AzureKinectSensor(config)
if not sensor.connect(device_id):
    raise RuntimeError('Failed to connect to sensor')

camera_ready = False
for _ in range(10):
    frame = sensor.capture_frame(enable_align_depth_to_color=True)
    if frame is not None:
        camera_ready = True
        break
    time.sleep(1.)

[Open3D INFO] AzureKinectSensor::Connect
[Open3D INFO] sensor_index 0
[Open3D INFO] Serial number: 000018922012
[Open3D INFO] Firmware build: Rel
[Open3D INFO] > Color: 1.6.110
[Open3D INFO] > Depth: 1.6.80[6109.7]


In [3]:
fps = 10
N_frames = 100
frames = []

if camera_ready:
    for _ in range(N_frames):
        frame = sensor.capture_frame(enable_align_depth_to_color=False)
        if frame is not None:
            frames.append((np.asarray(frame.color), np.asarray(frame.depth)))
        else:
            print("Empty frame observed!")
        time.sleep(1/fps)

In [4]:
save_gzip('frames.gzip', frames)